In [1]:
'''
Import Relevant Libraries
'''
import math
import time
import random
import numpy as np
from sympy import *

import ansatz
import ham_construction

import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 14

from qiskit import *
from qiskit.providers.ibmq import least_busy

from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import Pauli, SparsePauliOp

from qiskit.circuit import Parameter
from qiskit.circuit import QuantumCircuit
from qiskit.circuit import ParameterVector

from qiskit_aer.noise import NoiseModel
from qiskit.providers.fake_provider import FakeManila

from qiskit_algorithms.optimizers import SPSA, COBYLA, GradientDescent

from qiskit.primitives import Estimator as prim_Estimator
from qiskit_ibm_runtime import Estimator as run_Estimator
from qiskit_aer.primitives import Estimator as AerEstimator

from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options

In [2]:
'''
Define IBMQ service and find list of backends.
'''
#service = QiskitRuntimeService()
#service.backends()

'\nDefine IBMQ service and find list of backends.\n'

In [3]:
def rand_numb(low_in,high_in):
    '''
    Returns a random number between low_in and high_in, including both end points.
    '''
    return np.random.uniform(low=low_in, high=np.nextafter(high_in, np.inf))

In [4]:
def costf(params, qCirc, hamiltonian, estimator):
    '''
    Calculates the expectation value. 
    Inputs: 
        params: List of parameters
        qCirc: QuantumCircuit object of ansatz circuit
        hamiltonian: SparsePauliOp of the Hamiltonian
        estimator: Estimator object used to find cost function value
    '''
    job = estimator.run(qCirc, hamiltonian, params)
    result = job.result()
    return np.real(result.values[0])

In [46]:
def gradient(params, qCirc, hamiltonian, estimator):
    '''
    Calculates the gradient of the cost function. 
    Inputs: 
        params: List of parameters
        qCirc: QuantumCircuit object of ansatz circuit
        hamiltonian: SparsePauliOp of the Hamiltonian
        estimator: Estimator object used to find cost function value
    '''
    gradient = []
    for i in range(len(params)):
        param_plus = np.copy(params)
        param_minus = np.copy(params)
        param_plus[i] += np.pi/2
        param_minus[i] -= np.pi/2
        gradient.append(0.5*(np.real(estimator.run(qCirc, hamiltonian, param_plus).result().values[0]) - np.real(estimator.run(qCirc, hamiltonian, param_minus).result().values[0])))
        
    return np.array(gradient)

In [47]:
def callback(nfev, x, fx, stepsize, accepted = False):
    '''
    Callback function called automatically during optimization. Appends the expectation value, current iteration 
    and prints every iteration.
    '''
    callback_dict["iters"] += 1
    callback_dict["cost_history"].append(fx)
    print(
        "Iters. done: {} Current Cost: {}".format(callback_dict["iters"], callback_dict["cost_history"][-1]),
        end="\r",
        flush=True,
    )

In [48]:
'''
Parameters for n+C simulations. V0 in MeV, hOmega in MeV.
'''
#10C
nC = 10
V0 = -11.98302094
hOmega = 18.43541683
c = 0.03391566531

#12C
#nC = 12
#V0 = -4.934644166
#hOmega = 17.43690518
#c = 0.03493754913

#14C
#nC = 14
#V0 = -4.02317388
#hOmega = 16.62468545
#c = 0.04

#16C
#V0 = -2.790437277
#hOmega = 15.94535587
#c = 0.04526935265

#18C
#V0 = -2.950080624
#hOmega = 15.36500325
#c = 0.04725897921

In [49]:
'''
Pick number of qubits and encoding type.
    encType = 1 for Gray code 
    encType = 2 for Binary Coding
    encType = 3 for One-Hot Coding
'''
numQubit = 3
encType = 1

In [50]:
'''
Create ansatz for the given encoding type and number of qubits.
'''
numLayer = 4
qCirc = ansatz.create_RealHE_Ansatz(numQubit, numLayer)

In [51]:
'''
Pick k-value for number of off-diagonals. Total number of diagonals is 2K+1. 
Create Hamiltonian for given parameters and k.
'''
k=3
hamiltonian_matrix, hamiltonian_operator, true_value = ham_construction.createHamiltonianExp(encType, numQubit, k, V0, c, hOmega)
reversed_Pauli = [x[::-1].to_label() for x in hamiltonian_operator.paulis]
hamiltonian = SparsePauliOp(reversed_Pauli, hamiltonian_operator.coeffs)
print("True Value : ", true_value)
hamiltonian_matrix

True Value :  -0.49959533327750627


Matrix([
[     8.6752243630032,   -10.0017344437067, -0.00758032581873504, 0.000380822682485799,                   0,                   0,                   0,                   0],
[   -10.0017344437067,    25.5779915915888,    -18.2843349083236,  -0.0181900144640787, 0.00131920846948015,                   0,                   0,                   0],
[-0.00758032581873504,   -18.2843349083236,     42.4511482604176,    -26.5267634211751, -0.0309733180496362, 0.00309381809787112,                   0,                   0],
[0.000380822682485799, -0.0181900144640787,    -26.5267634211751,      59.299844032056,    -34.765823853738,  -0.044396944190762, 0.00596255543239386,                   0],
[                   0, 0.00131920846948015,  -0.0309733180496362,     -34.765823853738,    76.1292285690704,   -43.0071525168375, -0.0569182056225262,  0.0101830019925075],
[                   0,                   0,  0.00309381809787112,   -0.044396944190762,   -43.0071525168375,    92.94445153402

In [52]:
'''
Initialize random ansatz parameters. 
'''
paramProver = []
for i in range(0, len(qCirc.parameters)):
    paramProver = np.append(paramProver, np.array([rand_numb(0,2*np.pi)]))

In [53]:
def learning_rate_func():
    '''
    Defines learning rate function. Using a window size of 10, fit a straight line of the last 10 iterations. 
    If the slope is negative, increase learning rate lr to min(1.05*lr, lr_max).
    If the slope is positive, reduce learning rate lr to max(0.8*lr, lr_min).
    '''
    lr_min = 0.0001
    lr_max = 0.1
    def get_rate():
        lr = 0.01
        lr_wait = 0
        while True:
            lr_wait += 1
            if lr_wait >= 10:
                m, _ = np.polyfit(list(range(10)), callback_dict["cost_history"][-10:], 1)
                if m>0:
                    lr = max(0.8*lr, lr_min)
                else:
                    lr = min(1.05*lr, lr_max)
                lr_wait = 0
            yield lr
        
    return get_rate()

In [61]:
'''
Noiseless Simulation.
    1. Initiate empty callback_dict to track progress.
    2. Define estimator for noiseless simulation.
    3. Print initial cost.
    4. Wrap costf to implicitly inject the circuit, hamiltonian, and estimator.
    5. Train.
'''

callback_dict = {
    "iters": 0,
    "cost_history": [],
}
estimator = prim_Estimator()
noiselessParam = np.copy(paramProver)
count = 0
maxIterList = [500, 1500]
resampList = [2, 2]

for j in range(1, k+1):
    if (j%2==0):
        continue
    print(j)
    _, hamiltonian_operator, _ = ham_construction.createHamiltonianExp(encType, numQubit, j, V0, c, hOmega)
    reversed_Pauli = [x[::-1].to_label() for x in hamiltonian_operator.paulis]
    hamiltonian = SparsePauliOp(reversed_Pauli, hamiltonian_operator.coeffs)

    print("Initial Cost : ", costf(noiselessParam, qCirc, hamiltonian, estimator))

    autospsa = SPSA(maxiter=maxIterList[count], learning_rate=None, perturbation=None, callback=callback, resamplings=resampList[count])
    aug_costf = SPSA.wrap_function(costf, (qCirc, hamiltonian, estimator))
    obj = autospsa.minimize(fun=aug_costf, x0=noiselessParam)
    noiselessParam = obj.x
    count += 1

#print("Initial Cost : ", costf(noiselessParam, qCirc, hamiltonian, estimator))
#gd = GradientDescent(maxiter=1000, learning_rate=learning_rate_func, tol=1e-07, callback=callback, perturbation=None)
#aug_costf = GradientDescent.wrap_function(costf, (qCirc, hamiltonian, estimator))
#aug_gradient = GradientDescent.wrap_function(gradient, (qCirc, hamiltonian, estimator))
#obj = gd.minimize(fun=aug_costf, x0=noiselessParam)
#noiselessParam = obj.x

1
Initial Cost :  167.39854761368605
3ters. done: 500 Current Cost: 1.6881426851485148
Initial Cost :  1.0992373537163982


In [62]:
noiseless_callback_dict = callback_dict

In [65]:
'''
Shot Noise Simulation.
    1. Initiate empty callback_dict to track progress.
    2. Define estimator for shot noise simulation.
    3. Print initial cost.
    4. Wrap costf to implicitly inject the circuit, hamiltonian, and estimator.
    5. Train.
'''

callback_dict = {
    "iters": 0,
    "cost_history": [],
}

estimator = prim_Estimator(options={"shots": 1e3})
shotNoiseParam = np.copy(paramProver)
count = 0
maxIterList = [500, 1500]
resampList = [2, 2]

for j in range(1, k+1):
    if (j%2==0):
        continue
    print(j)

    _, hamiltonian_operator, _ = ham_construction.createHamiltonianExp(encType, numQubit, j, V0, c, hOmega)
    reversed_Pauli = [x[::-1].to_label() for x in hamiltonian_operator.paulis]
    hamiltonian = SparsePauliOp(reversed_Pauli, hamiltonian_operator.coeffs)

    print("Initial Cost : ", costf(shotNoiseParam, qCirc, hamiltonian, estimator))
    
    autospsa = SPSA(maxiter=maxIterList[count], learning_rate=None, perturbation=None, callback=callback, resamplings=resampList[count])
    aug_costf = SPSA.wrap_function(costf, (qCirc, hamiltonian, estimator))
    obj = autospsa.minimize(fun=aug_costf, x0=shotNoiseParam)
    shotNoiseParam = obj.x
    count += 1

#print("Initial Cost : ", costf(shotNoiseParam, qCirc, hamiltonian, estimator))
#gd = GradientDescent(maxiter=1000, learning_rate=learning_rate_func, tol=1e-07, callback=callback, perturbation=None)
#aug_costf = GradientDescent.wrap_function(costf, (qCirc, hamiltonian, estimator))
#aug_gradient = GradientDescent.wrap_function(gradient, (qCirc, hamiltonian, estimator))
#obj = gd.minimize(fun=aug_costf, x0=shotNoiseParam)
#shotNoiseParam = obj.x

1
Initial Cost :  166.59609749465164
3ters. done: 500 Current Cost: 28.876743098574178
Initial Cost :  27.00221833302317


In [66]:
shot_noise_callback_dict = callback_dict

In [67]:
'''
Noisy Simulation.
    1. Initiate empty callback_dict to track progress.
    2. Define estimator for noisy simulation.
    3. Print initial cost.
    4. Wrap costf to implicitly inject the circuit, hamiltonian, and estimator.
    5. Train.
'''
callback_dict = {
    "iters": 0,
    "cost_history": [],
}

device = FakeManila()
coupling_map = device.configuration().coupling_map
noise_model = NoiseModel.from_backend(device)
noisyParam = np.copy(paramProver)

estimator = AerEstimator(
    backend_options={
        "method": "density_matrix",
        "coupling_map": coupling_map,
        "noise_model": noise_model,
    },
    run_options={"shots": 1e3},
)

_, hamiltonian_operator, _ = ham_construction.createHamiltonianExp(encType, numQubit, 1, V0, c, hOmega)
reversed_Pauli = [x[::-1].to_label() for x in hamiltonian_operator.paulis]
hamiltonian = SparsePauliOp(reversed_Pauli, hamiltonian_operator.coeffs)

print("Initial Cost : ", costf(noisyParam, qCirc, hamiltonian, estimator))

autospsa = SPSA(maxiter=500, learning_rate=None, perturbation=None, callback=callback, resamplings=1)
aug_costf = SPSA.wrap_function(costf, (qCirc, hamiltonian, estimator))
obj = autospsa.minimize(fun=aug_costf, x0=noisyParam)
noisyParam = obj.x


_, hamiltonian_operator, _ = ham_construction.createHamiltonianExp(encType, numQubit, 3, V0, c, hOmega)
reversed_Pauli = [x[::-1].to_label() for x in hamiltonian_operator.paulis]
hamiltonian = SparsePauliOp(reversed_Pauli, hamiltonian_operator.coeffs)

estimator = AerEstimator(
    backend_options={
        "method": "density_matrix",
        "coupling_map": coupling_map,
        "noise_model": noise_model,
    },
    run_options={"shots": 2e4},
)

print("Initial Cost : ", costf(noisyParam, qCirc, hamiltonian, estimator))

autospsa = SPSA(maxiter=1500, learning_rate=None, perturbation=None, callback=callback, resamplings=2)
aug_costf = SPSA.wrap_function(costf, (qCirc, hamiltonian, estimator))
obj = autospsa.minimize(fun=aug_costf, x0=noisyParam)
noisyParam = obj.x

print("Noiseless value on noisy trained parameters : ", costf(noisyParam, qCirc, hamiltonian, prim_Estimator()))

Initial Cost :  175.7397633274418
Initial Cost :  21.773814585236508896846137881376
Noiseless value on noisy trained parameters :  0.7245751790592732


In [68]:
noisy_trained_value = costf(noisyParam, qCirc, hamiltonian, prim_Estimator())
noisy_callback_dict = callback_dict

In [ ]:
LB = least_busy(service.backends(filters=lambda x: not x.configuration().simulator))
session = Session(service=service, backend=LB.name)
estimator = run_Estimator(session=session)
quantumParam = np.copy(paramProver)

print("Initial Cost : ", costf(quantumParam, qCirc, hamiltonian, estimator))

In [ ]:
'''
Quantum Simulation.
    1. Initiate empty callback_dict to track progress.
    2. Define estimator for quantum simulation.
    3. Print initial cost.
    4. Wrap costf to implicitly inject the circuit, hamiltonian, and estimator.
    5. Train.
'''
callback_dict = {
    "iters": 0,
    "cost_history": [],
}

quan_options = Options(optimization_level=2, resilience_level=2)
quan_options.execution.shots = 4096
estimator = run_Estimator(session=session, options=quan_options)
quantumParam = np.copy(paramProver)

for j in range(1, k+1):
    if (j%2==0):
        continue
    print(j)

    _, hamiltonian_operator, _ = ham_construction.createHamiltonianExp(encType, numQubit, j, V0, c, hOmega)
    reversed_Pauli = [x[::-1].to_label() for x in hamiltonian_operator.paulis]
    hamiltonian = SparsePauliOp(reversed_Pauli, hamiltonian_operator.coeffs)
    
    print("Initial Cost : ", costf(quantumParam, qCirc, hamiltonian, estimator))
    
    autospsa = SPSA(maxiter=500, learning_rate = None, perturbation=None, callback=callback, resamplings=1)
    aug_costf = SPSA.wrap_function(costf, (qCirc, hamiltonian, estimator))
    obj = autospsa.minimize(fun=aug_costf, x0=quantumParam)
    quantumParam = obj.x
    
print("Noiseless value on quantum trained parameters : ", costf(quantumParam, qCirc, hamiltonian, prim_Estimator()))

In [ ]:
quantum_callback_dict = callback_dict

In [71]:
list_parameter = [V0, hOmega, c, numQubit, encType, k]

parameter_string = ""
for item in list_parameter:
    parameter_string += str(item)
    parameter_string += " "

print(parameter_string)

-4.02317388 16.62468545 0.04 8 3 3 


In [72]:
def writeToFile(filename, parameter_string, qt_back_name):
    '''
    Write the data to a text file to be used to plot the data. The format is as follows:
        1. Type of file - Fixed numQubit, Fixed k, Different backends
        2. parameter_string = "V0 hOmega c numQubit encType k"
        3. num_noiseless
        4. loss_noiseless
        5. num_shot_noise
        6. loss_shot_noise
        7. num_noisy
        8. loss_noisy
        9. noisy_trained_value
    '''
    file = open(filename, "w+")
    
    file.write("Fixed numQubit, Fixed K, Different backends\n")
    #file.write("Quantum backend used : " + qt_back_name + "\n")
    
    file.write(parameter_string + "\n")
    
    file.write(str(true_value) + "\n")
    
    file.write(str(noiseless_callback_dict["iters"]) + "\n")
    L = [str(i) + "\n" for i in noiseless_callback_dict["cost_history"]]
    file.writelines(L)
    
    file.write(str(shot_noise_callback_dict["iters"]) + "\n")
    L = [str(i) + "\n" for i in shot_noise_callback_dict["cost_history"]]
    file.writelines(L)
    
    file.write(str(noisy_callback_dict["iters"]) + "\n")
    L = [str(i) + "\n" for i in noisy_callback_dict["cost_history"]]
    file.writelines(L)
    file.write("Noisy Trained Value : " + str(noisy_trained_value))
    
    #L = [str(i) + "\n" for i in quantum_callback_dict["cost_history"]]
    #file.writelines(L)

    file.close()
    
writeToFile("Data/OH_C"+str(nC)+"_n_"+str(numQubit)+"_k_"+str(k)+"-"+time.strftime("%Y%m%d-%H%M%S"), parameter_string, "LB.name")